In [1]:
import py4dgeo
import laspy
from sklearn import cluster
from scipy import spatial
import numpy as np
import uuid
import datetime
import os
import sys
sys.path.insert(0, "../src")
from fourdgeo import projection
from fourdgeo import utilities

In [2]:
def cluster_m3c2_changes(significant_changes, dbscan_eps, min_cluster_size):
    """
    Cluster M3C2 changes using DBSCAN and return clusters with their properties.
    :param significant_changes: Array of significant changes with shape (n, 4) where n is the number of points.
    :param dbscan_eps: The maximum distance between two samples for one to be considered as
    :param min_cluster_size: The minimum number of samples in a cluster.
    :return: A list of clusters with their properties.
    """
    # DBSCAN clustering
    dbscan = cluster.DBSCAN(eps=dbscan_eps, min_samples=min_cluster_size)
    labels = dbscan.fit_predict(significant_changes[:, :-1])

    # Combine results and check that the labels are unique
    all_changes_with_labels = np.column_stack((significant_changes, labels))

    # Remove noise points (label -1)
    all_changes_with_labels = all_changes_with_labels[all_changes_with_labels[:, -1] != -1]
    return all_changes_with_labels

def extract_geoObjects_from_clusters(all_changes_with_labels, endDateTime_, filename_0, filename_1):
    """
    Extract observations from clusters of M3C2 changes.
    :param all_changes_with_labels: Array of significant changes with labels.
    :param endDateTime_: The end date and time of the observation.
    :param filename_0: The filename of the first epoch.
    :param filename_1: The filename of the second epoch.
    :return: A list of observations with geo objects.
    """
    # Extract unique cluster IDs and their counts
    cluster_ids, cluster_count = np.unique(all_changes_with_labels[:, -1],return_counts=True)

    # If no clusters found, continue to the next file
    if len(cluster_ids) == 0:
        print(f"No clusters found between {filename_0} and {filename_1}.")
        return

    # backgroundImageData_ = XXX

    geoObjects_ = []

    # If clusters found, print the largest and smallest cluster sizes    
    for cluster_id in range(len(cluster_ids)):
        xyz = all_changes_with_labels[all_changes_with_labels[:, -1] == cluster_ids[cluster_id], :3]
        m3c2_distances = all_changes_with_labels[all_changes_with_labels[:, -1] == cluster_ids[cluster_id], -2]
        convex_hull = spatial.ConvexHull(xyz)
        volume = convex_hull.volume
        area = convex_hull.area
        surface_to_volume_ratio = area / volume if volume > 0 else float('inf')
        m3c2_mean_distance = np.mean(np.abs(m3c2_distances))
        vertices_of_hull = convex_hull.points[convex_hull.vertices]

        # Create an geo object based on the cluster for the observations
        dateTime_ = endDateTime_
        # Create a unique ID for the cluster
        id_ = uuid.uuid4().hex
        # Type is "unknown", as we don't apply any classifier here
        type_ = "unknown"

        customEntityData_ = {
            "X_centroid": np.mean(xyz[:, 0]),
            "Y_centroid": np.mean(xyz[:, 1]),
            "Z_centroid": np.mean(xyz[:, 2]),
            "m3c2_magnitude_abs_average_per_cluster": m3c2_mean_distance,
            "volume": volume,
            "surface_area": area,
            "surface_to_volume_ratio": surface_to_volume_ratio,
            "cluster_size_points": int(cluster_count[cluster_id]),
            }
        
        geometry_ = {
            "type": "Polygon",
            "coordinates": vertices_of_hull.tolist()
        }

        geoObjects_.append({
            "id": id_,
            "type": type_,
            "dateTime": dateTime_,
            "geometry": geometry_,
            "customEntityData": customEntityData_
        })
    return geoObjects_

In [3]:
# Handle file download/reading here
infolder = r"/media/william/3b2c79aa-05fe-4d08-88bc-3669754da823/Ronny/low_res_bad_attr_clipped"
infolder = r"/media/william/FA2C-06E6/weekend/low_res_bad_attr_clipped"

m3c2_settings = {"cyl_radius":1,
                 "normal_radii":[1.0,],
                 "max_distance": 10.0,
                 "registration_error":0.025
                 }

#DBScan parameters
dbscan_eps = 1
min_cluster_size = 100

In [4]:
# observations = {"observations": []}

# # Gather & sort only the .laz files
# laz_files = sorted(f for f in os.listdir(infolder) if f.endswith('.laz'))

# # Walk through each consecutive pair
# for prev_fname, curr_fname in zip(laz_files, laz_files[1:]):
#     prev_path = os.path.join(infolder, prev_fname)
#     curr_path = os.path.join(infolder, curr_fname)

#     startDateTime = utilities.iso_timestamp(prev_fname)
#     endDateTime   = utilities.iso_timestamp(curr_fname)

#     # Load point clouds
#     epoch_0 = py4dgeo.read_from_las(prev_path)
#     epoch_1 = py4dgeo.read_from_las(curr_path)

#     # Compute M3C2
#     m3c2 = py4dgeo.M3C2(
#         epochs=(epoch_0, epoch_1),
#         corepoints=epoch_0.cloud,
#         cyl_radius=m3c2_settings["cyl_radius"],
#         normal_radii=m3c2_settings["normal_radii"],
#         max_distance=m3c2_settings["max_distance"],
#         registration_error=m3c2_settings["registration_error"],
#     )
#     distances, uncertainties = m3c2.run()

#     # Mask & stack only significant changes
#     mask = np.abs(distances) >= uncertainties["lodetection"]
#     if not mask.any():
#         print(f"No significant changes between {prev_fname} → {curr_fname}")
#         continue

#     significant_pts = epoch_0.cloud[mask]
#     significant_d  = distances[mask]
#     changes = np.column_stack((significant_pts, significant_d))

#     # Cluster & extract geoObjects
#     labeled = cluster_m3c2_changes(changes, dbscan_eps, min_cluster_size)
#     geoObjects = extract_geoObjects_from_clusters(labeled, endDateTime, prev_fname, curr_fname)

#     observations["observations"].append({
#         "backgroundImageData": {},
#         "startDateTime": startDateTime,
#         "endDateTime": endDateTime,
#         "geoObjects": geoObjects,
#     })

### Prepare the configuration file

In [5]:
configuration = {
    "project_setting": {
        "project_name": "Rockfall_monitoring",
        "output_folder": "./out",
        "temporal_format": "%y%m%d_%H%M%S",
        "silent_mode": True,
        "include_timestamp": False
    },
    "pc_projection": {
        "pc_path": "",
        "make_range_image": True,
        "make_color_image": False,
        "top_view": False,
        "save_rot_pc": False,
        "resolution_cm": 12.5,
        "camera_position": [
            0.0,
            0.0,
            0.0
        ],
        "rgb_light_intensity": 100,
        "range_light_intensity": 10,
        "epsg": None
    }
}

### Generating the background images

In [6]:
images = []
list_background_projections = []
laz_files = sorted(f for f in os.listdir(infolder) if f.endswith('.laz'))

for enum, laz_file in enumerate(laz_files):
    pc = os.path.join(infolder, laz_file)
    lf = laspy.read(pc)
    configuration['pc_projection']['pc_path'] = pc
    background_projection = projection.PCloudProjection(
        configuration = configuration,
        project_name = configuration['project_setting']['project_name'],
        projected_image_folder = configuration['project_setting']['output_folder'],
    )
    # First projection
    if enum == 0:
        (
            ref_h_fov, ref_v_fov, ref_anchor_point_xyz, 
            ref_h_img_res, ref_v_img_res
        ) = background_projection.project_pc(buffer_m = 0.5)
    # Next projections using reference data
    else:
        background_projection.project_pc(
            ref_theta=ref_h_fov[0],
            ref_phi=ref_v_fov[0],
            ref_anchor_point_xyz=None,
            ref_h_fov=ref_h_fov,
            ref_v_fov=ref_v_fov,
            ref_h_img_res=ref_h_img_res,
            ref_v_img_res=ref_v_img_res
        )
    outfile = f"out/Rockfall_monitoring_RangeImage_{enum}_{enum+1}.tif"
    try:
        os.rename("out/Rockfall_monitoring_RangeImage.tif", outfile)
    except FileExistsError:
        os.remove(outfile)
        os.rename("out/Rockfall_monitoring_RangeImage.tif", outfile)
    images.append(outfile)

    background_projection.bg_image_filename[0] = outfile
    list_background_projections.append(background_projection)

In [7]:
from PIL import Image
from IPython.display import HTML

gif_path = "../docs/img/rockfall_projections.gif"
frames = [Image.open(img).convert("RGB") for img in images]

frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=500,
    loop=0
)

HTML(f"""
<img src="{gif_path}" style="height:5in;" />
""")

In [8]:
observations = {'observations': [{'backgroundImageData': {},
   'startDateTime': '2024-08-25T22:00:05',
   'endDateTime': '2024-08-25T23:00:05',
   'geoObjects': None},
  {'backgroundImageData': {},
   'startDateTime': '2024-08-25T23:00:05',
   'endDateTime': '2024-08-26T00:00:05',
   'geoObjects': None},
  {'backgroundImageData': {},
   'startDateTime': '2024-08-26T00:00:05',
   'endDateTime': '2024-08-26T01:00:06',
   'geoObjects': [{'id': '1399a1a27cad42d7aa21e64abd241f4e',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-258.019, 158.27100000000002, 5.363],
       [-258.015, 158.19400000000002, 5.831],
       [-258.005, 158.187, 5.751],
       [-258.005, 158.18800000000002, 5.672],
       [-258.397, 158.428, 5.36],
       [-258.177, 158.293, 5.04],
       [-258.129, 158.151, 5.9510000000000005],
       [-258.677, 158.487, 5.244],
       [-258.25100000000003, 158.225, 5.156],
       [-258.298, 158.255, 5.0],
       [-258.615, 158.356, 6.099],
       [-258.32, 158.17600000000002, 6.009],
       [-259.543, 158.731, 5.434],
       [-259.471, 158.687, 5.272],
       [-259.38, 158.52700000000002, 6.253],
       [-259.473, 158.584, 6.098],
       [-259.046, 158.323, 4.811],
       [-258.974, 158.214, 4.847],
       [-260.13100000000003, 158.64000000000001, 5.308],
       [-259.866, 158.478, 5.15],
       [-260.162, 158.534, 6.9750000000000005],
       [-260.031, 158.454, 6.892],
       [-259.777, 158.3, 5.12],
       [-260.225, 158.491, 7.009],
       [-260.595, 158.431, 6.273],
       [-260.55400000000003, 158.406, 6.111],
       [-260.559, 158.409, 6.027],
       [-260.64300000000003, 158.46, 5.547],
       [-261.731, 158.92000000000002, 6.343],
       [-261.337, 158.681, 5.849],
       [-261.803, 158.873, 6.354],
       [-261.598, 158.673, 7.025],
       [-261.765, 158.775, 6.625],
       [-261.791, 158.793, 6.465],
       [-261.66700000000003, 158.607, 6.676]]},
     'customEntityData': {'X_centroid': np.float64(-259.8313391304348),
      'Y_centroid': np.float64(158.48924782608697),
      'Z_centroid': np.float64(5.991530434782609),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(1.0733661294671373),
      'volume': 1.2427361073333316,
      'surface_area': 11.436782819482161,
      'surface_to_volume_ratio': 9.20290538916042,
      'cluster_size_points': 230}},
    {'id': 'e4005fae9bf94091852d0d1d7cac9700',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-265.481, 161.399, 9.665000000000001],
       [-265.105, 161.171, 9.48],
       [-265.208, 161.234, 9.078],
       [-265.098, 161.167, 8.993],
       [-265.058, 161.142, 8.909],
       [-265.35200000000003, 161.242, 9.754],
       [-265.108, 161.095, 8.934000000000001],
       [-265.786, 161.507, 8.707],
       [-265.485, 161.198, 8.474],
       [-265.593, 161.171, 10.048],
       [-265.384, 161.04500000000002, 9.794],
       [-265.662, 161.214, 8.407],
       [-266.139, 161.317, 8.349],
       [-266.468, 161.422, 9.949],
       [-266.568, 161.483, 9.796],
       [-266.171, 161.244, 8.464],
       [-266.431, 161.311, 10.019],
       [-267.082, 161.591, 8.846],
       [-266.378, 161.08, 10.083],
       [-266.426, 161.11, 9.017],
       [-266.556, 161.119, 10.154],
       [-267.468, 161.671, 9.282],
       [-267.375, 161.615, 9.117],
       [-267.527, 161.58, 9.254],
       [-267.237, 161.303, 9.411],
       [-267.535, 161.413, 9.549]]},
     'customEntityData': {'X_centroid': np.float64(-266.1051468531468),
      'Y_centroid': np.float64(161.27732167832167),
      'Z_centroid': np.float64(9.41537762237762),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.5632060607257144),
      'volume': 0.874495007833308,
      'surface_area': 7.1769161467972555,
      'surface_to_volume_ratio': 8.206926377520595,
      'cluster_size_points': 143}},
    {'id': '621bd231ea7a42dfa6576febc182cb13',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-261.849, 158.827, 6.9510000000000005],
       [-261.889, 158.851, 6.383],
       [-261.897, 158.855, 6.3020000000000005],
       [-261.88100000000003, 158.737, 6.765000000000001],
       [-263.048, 159.352, 8.284],
       [-262.403, 158.96200000000002, 6.5680000000000005],
       [-262.564, 158.859, 7.8100000000000005],
       [-262.757, 158.892, 6.548],
       [-262.022, 158.448, 6.453],
       [-263.50600000000003, 159.276, 8.985],
       [-263.303, 158.929, 8.627],
       [-263.267, 158.907, 8.545],
       [-263.069, 158.786, 7.971],
       [-263.468, 158.959, 8.673],
       [-264.836, 159.502, 8.714],
       [-265.342, 159.519, 7.8100000000000005],
       [-265.635, 159.40800000000002, 9.234],
       [-264.895, 158.965, 7.508],
       [-265.81600000000003, 159.398, 8.047],
       [-266.321, 159.615, 8.832],
       [-265.992, 159.345, 9.290000000000001],
       [-266.45300000000003, 159.621, 8.818],
       [-266.073, 159.394, 8.229000000000001],
       [-265.577, 159.097, 7.8020000000000005],
       [-266.409, 159.467, 9.257],
       [-266.15500000000003, 159.316, 8.266],
       [-266.488, 159.423, 9.355],
       [-266.669, 159.532, 9.037],
       [-266.619, 159.434, 9.345],
       [-266.731, 159.499, 9.021],
       [-266.145, 159.15, 8.752],
       [-266.38100000000003, 159.29, 8.44],
       [-266.435, 159.204, 8.565],
       [-266.408, 159.18800000000002, 8.482]]},
     'customEntityData': {'X_centroid': np.float64(-264.238488700565),
      'Y_centroid': np.float64(159.1311440677966),
      'Z_centroid': np.float64(8.024302259887007),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.7916445742482292),
      'volume': 2.9302886854999897,
      'surface_area': 17.319365292482566,
      'surface_to_volume_ratio': 5.910463831834847,
      'cluster_size_points': 354}},
    {'id': 'c445fe733b5f482c9da680dedcd83194',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-258.089, 156.44, 4.519],
       [-258.371, 156.611, 4.448],
       [-258.113, 156.362, 4.471],
       [-258.23, 156.435, 4.397],
       [-258.088, 156.263, 6.128],
       [-258.021, 156.221, 4.771],
       [-258.288, 156.383, 4.381],
       [-258.098, 156.076, 5.246],
       [-258.445, 156.096, 4.329],
       [-258.347, 155.93800000000002, 4.3340000000000005],
       [-260.288, 156.916, 5.276],
       [-260.188, 156.856, 5.19],
       [-259.353, 156.267, 4.657],
       [-258.387, 155.62, 5.566],
       [-259.072, 155.751, 4.531],
       [-263.128, 157.229, 6.441],
       [-263.204, 156.994, 6.363],
       [-263.722, 157.207, 8.564],
       [-264.825, 157.573, 7.2620000000000005],
       [-263.701, 156.558, 6.596],
       [-265.95300000000003, 157.763, 7.843],
       [-266.087, 157.0, 7.6770000000000005],
       [-265.908, 156.895, 7.591],
       [-268.082, 157.716, 8.704],
       [-267.936, 157.63, 8.621],
       [-268.76300000000003, 158.012, 9.097],
       [-270.523, 157.612, 9.792],
       [-271.866, 158.0, 10.525],
       [-271.88, 157.945, 10.518],
       [-273.892, 158.697, 13.622],
       [-274.772, 158.744, 12.168000000000001],
       [-274.636, 158.665, 12.075000000000001],
       [-275.076, 158.797, 14.393],
       [-274.811, 157.969, 11.921],
       [-277.536, 159.059, 13.636000000000001],
       [-277.669, 159.03300000000002, 13.678],
       [-277.281, 158.52, 13.285],
       [-277.401, 158.40200000000002, 13.31],
       [-277.583, 157.619, 13.335],
       [-281.53000000000003, 159.565, 17.856],
       [-282.647, 159.90800000000002, 17.774],
       [-283.41700000000003, 159.881, 16.881],
       [-279.836, 157.864, 14.545],
       [-285.994, 160.131, 18.244],
       [-285.874, 160.064, 18.154],
       [-289.846, 160.366, 22.094],
       [-290.45, 160.601, 22.34],
       [-294.217, 161.397, 22.868000000000002],
       [-292.652, 160.405, 23.54],
       [-292.671, 160.317, 23.474],
       [-294.758, 161.387, 23.188],
       [-294.842, 161.119, 24.446],
       [-294.873, 161.024, 24.435000000000002],
       [-295.351, 161.286, 23.587],
       [-295.148, 161.06900000000002, 24.345],
       [-295.375, 161.19400000000002, 23.735],
       [-294.644, 160.796, 22.969],
       [-295.184, 160.881, 24.357],
       [-295.365, 160.981, 23.396],
       [-294.799, 160.673, 23.088],
       [-295.563, 160.985, 23.699],
       [-295.35200000000003, 160.871, 23.419],
       [-294.187, 160.169, 22.843]]},
     'customEntityData': {'X_centroid': np.float64(-276.66497645799893),
      'Y_centroid': np.float64(158.6653691813804),
      'Z_centroid': np.float64(14.081360888175492),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(1.0404311325069902),
      'volume': 86.74677968066635,
      'surface_area': 238.3900924506905,
      'surface_to_volume_ratio': 2.7481146081532475,
      'cluster_size_points': 3738}},
    {'id': 'c95b5eed435442538abece49f15ec7e3',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-258.185, 154.547, 4.134],
       [-258.12, 154.508, 4.057],
       [-258.008, 154.353, 3.99],
       [-258.495, 154.46, 6.09],
       [-258.276, 154.33, 5.926],
       [-258.228, 154.20000000000002, 5.956],
       [-258.028, 154.079, 5.472],
       [-259.148, 154.749, 4.465],
       [-258.356, 154.216, 6.025],
       [-258.038, 154.026, 5.625],
       [-259.44, 154.749, 4.523],
       [-262.559, 155.205, 7.755],
       [-258.007, 152.077, 5.2],
       [-258.25600000000003, 152.224, 3.786],
       [-258.42900000000003, 152.121, 6.01],
       [-258.003, 151.871, 4.493],
       [-264.139, 155.296, 6.6000000000000005],
       [-265.919, 154.821, 7.3100000000000005],
       [-258.101, 150.101, 5.573],
       [-258.17, 150.04500000000002, 5.673],
       [-258.004, 149.949, 5.037],
       [-258.974, 150.059, 6.186],
       [-270.128, 156.241, 9.479000000000001],
       [-266.577, 154.189, 7.486],
       [-264.332, 152.796, 6.335],
       [-266.967, 153.572, 7.572],
       [-274.449, 157.114, 11.664],
       [-258.03000000000003, 147.52100000000002, 3.431],
       [-258.06600000000003, 147.471, 3.39],
       [-258.139, 147.409, 3.379],
       [-258.385, 147.457, 3.4130000000000003],
       [-275.689, 157.132, 14.292],
       [-276.575, 157.639, 12.909],
       [-276.09000000000003, 157.268, 12.507],
       [-276.5, 157.419, 12.747],
       [-259.261, 147.411, 3.725],
       [-272.467, 154.715, 10.364],
       [-264.088, 149.959, 5.8870000000000005],
       [-261.106, 147.999, 7.224],
       [-258.346, 145.815, 5.5760000000000005],
       [-273.744, 154.204, 10.9],
       [-281.01800000000003, 158.234, 15.336],
       [-262.927, 148.055, 5.206],
       [-281.504, 158.216, 16.833000000000002],
       [-258.014, 145.024, 4.898],
       [-282.344, 157.51, 18.159],
       [-282.394, 157.417, 18.22],
       [-266.384, 148.403, 6.729],
       [-285.926, 157.909, 17.775000000000002],
       [-289.123, 158.90200000000002, 20.684],
       [-290.127, 158.635, 22.03],
       [-290.24, 158.611, 20.215],
       [-287.09000000000003, 156.681, 18.058],
       [-274.695, 149.63400000000001, 10.721],
       [-292.532, 158.742, 21.494],
       [-292.51, 158.66, 23.306],
       [-293.032, 158.946, 21.943],
       [-290.057, 157.107, 19.728],
       [-293.682, 158.995, 22.392],
       [-290.0, 157.006, 19.676000000000002],
       [-293.835, 158.769, 23.648],
       [-293.894, 158.80100000000002, 23.565],
       [-294.008, 158.863, 23.393],
       [-294.123, 158.925, 23.14],
       [-294.342, 158.919, 23.172],
       [-294.35200000000003, 158.924, 23.092],
       [-278.999, 150.197, 12.922],
       [-258.382, 138.846, 4.924],
       [-258.003, 138.643, 3.755],
       [-258.007, 138.19, 3.267],
       [-258.015, 138.014, 2.989],
       [-258.03000000000003, 138.022, 2.912],
       [-258.037, 137.35, 3.779],
       [-258.015, 137.339, 3.012],
       [-258.012, 136.969, 3.289],
       [-258.606, 136.853, 2.92],
       [-277.51800000000003, 146.654, 11.658],
       [-298.456, 157.63, 26.587],
       [-258.234, 135.773, 3.858],
       [-258.02, 135.659, 3.009],
       [-299.494, 157.266, 27.238],
       [-300.238, 157.529, 27.295],
       [-258.85, 135.836, 4.549],
       [-288.767, 151.43200000000002, 18.062],
       [-300.896, 157.577, 27.114],
       [-300.891, 157.467, 27.482],
       [-301.458, 157.764, 26.275000000000002],
       [-300.96500000000003, 157.437, 27.439],
       [-301.609, 157.543, 26.191],
       [-301.563, 157.519, 26.097],
       [-301.601, 157.243, 26.02],
       [-258.081, 134.5, 3.114],
       [-298.05, 154.998, 26.539],
       [-258.236, 134.316, 3.111],
       [-258.425, 134.318, 3.5620000000000003],
       [-260.666, 135.483, 3.592],
       [-258.69100000000003, 134.358, 2.894],
       [-292.153, 151.656, 19.826],
       [-301.551, 155.68, 28.179000000000002],
       [-267.346, 137.792, 6.111],
       [-280.98900000000003, 143.964, 13.049],
       [-281.789, 144.20000000000002, 13.485],
       [-264.874, 134.747, 6.897],
       [-301.783, 153.38400000000001, 27.962],
       [-263.95300000000003, 133.933, 5.453],
       [-263.906, 133.849, 5.223],
       [-273.943, 138.82500000000002, 9.027000000000001],
       [-264.233, 133.804, 5.226],
       [-266.72700000000003, 135.01500000000001, 5.939],
       [-265.81, 134.433, 5.581],
       [-265.954, 134.349, 5.641],
       [-273.242, 137.93, 8.586],
       [-265.738, 134.143, 6.3870000000000005],
       [-265.938, 134.154, 5.73],
       [-266.038, 134.136, 5.78],
       [-266.84000000000003, 134.34, 6.4750000000000005],
       [-267.117, 134.479, 6.245],
       [-291.318, 146.566, 18.701],
       [-275.151, 137.524, 9.724],
       [-305.472, 152.054, 26.865000000000002],
       [-274.92900000000003, 136.714, 9.947000000000001],
       [-303.009, 149.92000000000002, 27.619],
       [-302.481, 149.66, 27.311],
       [-279.184, 138.151, 12.037],
       [-286.597, 141.383, 15.707],
       [-280.015, 138.139, 13.288],
       [-286.673, 141.212, 15.801],
       [-286.567, 140.802, 17.327],
       [-292.702, 143.118, 20.79],
       [-287.914, 140.782, 16.806],
       [-305.21, 149.121, 27.637],
       [-306.6, 149.802, 27.668],
       [-306.675, 149.839, 27.411],
       [-306.562, 149.712, 27.82],
       [-304.551, 148.608, 25.797],
       [-306.583, 149.498, 27.779],
       [-306.66, 149.328, 27.693],
       [-306.624, 149.214, 27.608],
       [-305.3, 148.201, 27.465],
       [-305.354, 148.102, 27.443],
       [-306.373, 148.407, 27.082],
       [-304.617, 147.558, 25.666],
       [-306.193, 147.814, 27.080000000000002],
       [-306.115, 147.695, 27.039],
       [-304.641, 146.984, 26.721],
       [-305.919, 147.601, 26.575],
       [-305.793, 147.541, 26.466],
       [-306.107, 147.472, 26.816],
       [-306.116, 147.477, 26.731],
       [-305.931, 147.093, 26.729],
       [-305.12, 146.625, 25.919]]},
     'customEntityData': {'X_centroid': np.float64(-279.3063575929691),
      'Y_centroid': np.float64(148.32354882454754),
      'Z_centroid': np.float64(14.137747752440808),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.9524305817732593),
      'volume': 2377.0226303891677,
      'surface_area': 2089.6913825286774,
      'surface_to_volume_ratio': 0.8791213662894543,
      'cluster_size_points': 59509}},
    {'id': '2f0992c23a974828ba473c5c155dca6e',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-274.999, 162.673, 13.115],
       [-275.338, 162.755, 13.188],
       [-275.004, 162.558, 13.088000000000001],
       [-275.23, 162.691, 13.015],
       [-275.283, 162.618, 14.085],
       [-275.234, 162.592, 12.993],
       [-275.25600000000003, 162.53, 14.122],
       [-276.163, 162.94, 14.653],
       [-275.38, 162.478, 14.186],
       [-275.244, 162.399, 12.997],
       [-276.019, 162.68800000000002, 14.593],
       [-275.446, 162.35, 12.959],
       [-276.274, 162.71200000000002, 14.68],
       [-275.562, 162.29500000000002, 13.045],
       [-276.246, 162.604, 14.61],
       [-276.705, 162.875, 13.790000000000001],
       [-276.6, 162.811, 13.697000000000001],
       [-276.104, 162.431, 14.448],
       [-275.39300000000003, 162.013, 13.824],
       [-275.313, 161.966, 13.655000000000001],
       [-275.307, 161.963, 13.567],
       [-275.897, 162.31, 13.262],
       [-276.311, 162.445, 14.513],
       [-276.547, 162.584, 14.348],
       [-276.569, 162.489, 13.627]]},
     'customEntityData': {'X_centroid': np.float64(-275.75011926605504),
      'Y_centroid': np.float64(162.5110550458716),
      'Z_centroid': np.float64(13.772862385321101),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.5188641338014451),
      'volume': 0.8353865046666555,
      'surface_area': 5.3236945529809585,
      'surface_to_volume_ratio': 6.372732290073651,
      'cluster_size_points': 109}},
    {'id': '91e45049c48a4c0fa6f63f8c0fb4f7c3',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-272.794, 160.20600000000002, 12.568],
       [-272.821, 160.223, 12.485],
       [-273.124, 160.276, 13.123000000000001],
       [-273.062, 160.239, 13.040000000000001],
       [-272.916, 160.154, 12.528],
       [-273.574, 160.541, 11.888],
       [-273.27, 160.363, 11.624],
       [-273.225, 160.244, 13.175],
       [-273.092, 160.166, 12.993],
       [-273.995, 160.697, 12.118],
       [-273.39300000000003, 160.344, 11.677],
       [-273.975, 160.611, 13.428],
       [-273.13800000000003, 160.122, 12.468],
       [-273.44100000000003, 160.3, 11.732000000000001],
       [-274.106, 160.56300000000002, 13.922],
       [-274.337, 160.316, 14.040000000000001],
       [-274.241, 160.26, 13.795],
       [-275.007, 160.709, 12.564],
       [-275.81, 161.101, 14.955],
       [-274.442, 160.303, 14.045],
       [-274.357, 160.255, 12.702],
       [-275.081, 160.553, 14.812000000000001],
       [-276.05, 161.024, 13.189],
       [-276.397, 161.151, 13.405000000000001],
       [-275.858, 160.53900000000002, 15.143],
       [-276.335, 160.701, 13.212],
       [-277.921, 160.744, 14.071],
       [-277.56600000000003, 160.53900000000002, 13.88],
       [-279.402, 161.204, 17.031],
       [-279.56, 161.296, 16.866],
       [-279.577, 161.306, 16.782],
       [-279.726, 161.193, 17.146],
       [-279.726, 161.094, 17.341],
       [-280.19, 161.363, 16.601],
       [-279.799, 161.055, 17.314],
       [-280.919, 161.476, 15.858],
       [-280.831, 161.425, 15.763],
       [-280.999, 161.458, 15.909],
       [-281.426, 161.181, 17.119],
       [-281.589, 161.274, 17.043],
       [-281.582, 161.27100000000002, 16.185],
       [-281.56600000000003, 161.184, 16.928],
       [-280.797, 160.744, 16.622],
       [-281.402, 161.091, 16.061]]},
     'customEntityData': {'X_centroid': np.float64(-276.91485731132076),
      'Y_centroid': np.float64(160.75147995283018),
      'Z_centroid': np.float64(14.654399764150943),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.8310643109939657),
      'volume': 9.284877036833258,
      'surface_area': 41.621027645241334,
      'surface_to_volume_ratio': 4.482668696648329,
      'cluster_size_points': 848}},
    {'id': 'f27d2f12bd864f0f990522bce60a708e',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-281.833, 161.41400000000002, 16.714],
       [-281.995, 161.507, 16.638],
       [-282.02, 161.443, 17.635],
       [-281.597, 161.201, 17.183],
       [-281.935, 161.39600000000002, 16.525],
       [-282.103, 161.372, 17.862000000000002],
       [-281.706, 161.14600000000002, 17.154],
       [-281.689, 161.136, 17.064],
       [-282.658, 161.691, 16.868000000000002],
       [-282.134, 161.392, 16.571],
       [-281.848, 161.127, 17.102],
       [-281.916, 161.166, 17.016000000000002],
       [-283.111, 161.848, 17.088],
       [-282.38100000000003, 161.356, 17.939],
       [-282.081, 161.186, 17.404],
       [-282.918, 161.66400000000002, 16.939],
       [-282.446, 161.395, 16.731],
       [-282.75600000000003, 161.454, 18.208000000000002],
       [-282.57800000000003, 161.352, 17.935],
       [-282.738, 161.445, 16.834],
       [-282.896, 161.433, 18.156],
       [-282.966, 161.474, 16.951],
       [-283.75100000000003, 161.723, 18.5],
       [-283.427, 161.53900000000002, 17.185],
       [-284.414, 162.002, 18.497],
       [-284.389, 161.989, 17.807],
       [-284.306, 161.942, 17.715],
       [-283.61, 161.546, 17.327],
       [-283.433, 161.445, 17.233],
       [-284.044, 161.707, 18.617],
       [-283.584, 161.446, 17.552],
       [-284.1, 161.612, 18.565],
       [-284.38100000000003, 161.68200000000002, 18.655],
       [-284.422, 161.632, 18.252],
       [-284.519, 161.55100000000002, 18.505]]},
     'customEntityData': {'X_centroid': np.float64(-283.1086647398844),
      'Y_centroid': np.float64(161.5172774566474),
      'Z_centroid': np.float64(17.71605202312139),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.7696776635575184),
      'volume': 1.0082795101666804,
      'surface_area': 8.015285071108858,
      'surface_to_volume_ratio': 7.949467375156555,
      'cluster_size_points': 173}},
    {'id': 'd2b7cd6278af4fc091c4c84849d64f64',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-285.409, 161.762, 19.221],
       [-285.389, 161.751, 19.137],
       [-285.466, 161.70000000000002, 19.061],
       [-285.472, 161.703, 18.978],
       [-285.537, 161.74, 18.896],
       [-285.666, 161.742, 19.389],
       [-285.52500000000003, 161.662, 19.123],
       [-285.537, 161.669, 18.947],
       [-286.469, 162.197, 18.921],
       [-285.854, 161.849, 18.62],
       [-286.48, 162.073, 20.187],
       [-285.74, 161.656, 19.18],
       [-286.562, 162.122, 18.887],
       [-286.377, 162.016, 18.788],
       [-286.198, 161.916, 18.693],
       [-286.395, 161.93, 20.305],
       [-287.073, 162.315, 19.397000000000002],
       [-286.282, 161.78900000000002, 18.719],
       [-286.875, 161.83, 18.995],
       [-286.588, 161.668, 18.885],
       [-287.121, 161.83, 20.297],
       [-288.771, 162.593, 20.311],
       [-288.707, 162.55700000000002, 20.219],
       [-289.15500000000003, 162.27100000000002, 21.900000000000002],
       [-288.75, 162.04500000000002, 21.61],
       [-288.823, 162.012, 21.487000000000002],
       [-290.012, 162.68, 20.88],
       [-289.401, 162.338, 20.48],
       [-288.413, 161.785, 19.89],
       [-289.652, 162.364, 21.983],
       [-290.51300000000003, 162.744, 21.284],
       [-290.494, 162.734, 21.198],
       [-289.791, 162.252, 22.085],
       [-290.178, 162.352, 22.149],
       [-290.307, 162.42600000000002, 21.015],
       [-290.277, 162.311, 22.19],
       [-290.523, 162.45000000000002, 21.235],
       [-289.48900000000003, 161.875, 20.47],
       [-290.461, 162.32500000000002, 21.97],
       [-290.615, 162.412, 21.802],
       [-290.40000000000003, 162.173, 21.052],
       [-290.283, 162.108, 20.956],
       [-290.028, 161.965, 20.853]]},
     'customEntityData': {'X_centroid': np.float64(-288.11577810650886),
      'Y_centroid': np.float64(162.10728402366865),
      'Z_centroid': np.float64(20.369825443786983),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(1.0019658319075873),
      'volume': 3.019144622833356,
      'surface_area': 18.499516767105714,
      'surface_to_volume_ratio': 6.127403313904386,
      'cluster_size_points': 338}},
    {'id': 'ab23e1d68a134349bd83324fd9a306e4',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-292.193, 162.695, 22.79],
       [-292.733, 162.995, 22.478],
       [-292.214, 162.58700000000002, 23.04],
       [-292.168, 162.562, 22.952],
       [-292.51, 162.754, 22.262],
       [-292.736, 162.761, 23.436],
       [-292.57800000000003, 162.672, 23.343],
       [-292.358, 162.55, 23.148],
       [-292.325, 162.532, 23.053],
       [-292.573, 162.673, 22.283],
       [-292.639, 162.641, 22.311],
       [-292.817, 162.617, 23.457],
       [-293.428, 162.958, 22.786],
       [-293.161, 162.81, 22.596],
       [-293.14300000000003, 162.70000000000002, 23.681],
       [-293.654, 162.985, 22.922],
       [-293.81600000000003, 162.859, 23.947],
       [-294.007, 162.966, 23.076],
       [-293.764, 162.738, 23.963],
       [-294.096, 162.924, 23.092],
       [-292.99, 162.312, 22.477],
       [-293.892, 162.723, 23.932000000000002],
       [-293.87600000000003, 162.716, 22.948],
       [-294.14, 162.743, 23.133],
       [-293.815, 162.562, 22.926000000000002],
       [-294.052, 162.592, 23.062],
       [-293.866, 162.49, 22.962],
       [-293.658, 162.375, 22.857]]},
     'customEntityData': {'X_centroid': np.float64(-293.16931775700937),
      'Y_centroid': np.float64(162.7120373831776),
      'Z_centroid': np.float64(23.037841121495326),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.8132412586860251),
      'volume': 0.6278946891666815,
      'surface_area': 5.0566877695898915,
      'surface_to_volume_ratio': 8.053401082753128,
      'cluster_size_points': 107}},
    {'id': '056290775b544cc0ae64917e6f1c5111',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-294.838, 161.005, 24.529],
       [-295.095, 161.03900000000002, 24.782],
       [-295.208, 161.101, 24.609],
       [-295.135, 160.966, 24.848],
       [-295.193, 160.885, 24.892],
       [-295.142, 160.857, 24.447],
       [-296.178, 161.423, 23.996000000000002],
       [-296.33, 161.33100000000002, 25.244],
       [-296.452, 161.4, 24.091],
       [-295.635, 160.95600000000002, 23.672],
       [-295.664, 160.851, 23.552],
       [-296.76, 161.268, 24.195],
       [-297.04200000000003, 161.291, 25.646],
       [-297.773, 161.58, 24.921],
       [-297.766, 161.576, 24.827],
       [-297.681, 161.53, 24.734],
       [-296.873, 161.093, 24.219],
       [-297.269, 161.238, 25.746000000000002],
       [-297.19, 161.195, 25.650000000000002],
       [-297.835, 161.546, 25.077],
       [-297.82, 161.538, 24.810000000000002],
       [-296.658, 160.909, 24.083000000000002],
       [-297.665, 161.314, 25.737000000000002],
       [-297.782, 161.377, 25.564],
       [-297.37, 161.155, 25.439],
       [-297.932, 161.46, 25.127],
       [-297.961, 161.475, 25.04],
       [-297.85, 161.31900000000002, 25.452],
       [-298.05, 161.429, 24.93],
       [-298.052, 161.357, 24.974],
       [-297.948, 161.16400000000002, 24.849],
       [-297.777, 161.072, 24.745]]},
     'customEntityData': {'X_centroid': np.float64(-296.6386375),
      'Y_centroid': np.float64(161.19785000000002),
      'Z_centroid': np.float64(24.79639375),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(1.0365744811895063),
      'volume': 1.207735657333338,
      'surface_area': 9.144617859042084,
      'surface_to_volume_ratio': 7.57170478781198,
      'cluster_size_points': 160}},
    {'id': '9b985c2a6a6c42b38ab8cfae25a72eed',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-297.785, 161.379, 25.923000000000002],
       [-297.82, 161.398, 25.84],
       [-298.308, 161.567, 25.85],
       [-298.47, 161.656, 25.239],
       [-298.458, 161.649, 25.152],
       [-298.362, 161.598, 25.05],
       [-298.05, 161.354, 26.138],
       [-297.964, 161.308, 25.86],
       [-298.175, 161.198, 25.628],
       [-299.312, 161.812, 25.627],
       [-299.213, 161.75900000000001, 25.525000000000002],
       [-298.365, 161.30100000000002, 25.101],
       [-299.099, 161.481, 26.88],
       [-299.861, 161.894, 25.958000000000002],
       [-299.629, 161.679, 26.854],
       [-299.962, 161.772, 26.804000000000002],
       [-299.681, 161.622, 25.696],
       [-299.534, 161.543, 25.597],
       [-299.211, 161.221, 26.546],
       [-300.07, 161.685, 26.080000000000002],
       [-299.14300000000003, 161.186, 25.373],
       [-299.46, 161.29, 26.364],
       [-299.36400000000003, 161.24, 25.45],
       [-298.838, 160.95600000000002, 25.32],
       [-299.847, 161.40200000000002, 25.937],
       [-299.709, 161.328, 25.658]]},
     'customEntityData': {'X_centroid': np.float64(-299.06798529411765),
      'Y_centroid': np.float64(161.48562500000003),
      'Z_centroid': np.float64(25.886345588235294),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.7401630230748453),
      'volume': 0.9730775776666488,
      'surface_area': 6.7258448328476,
      'surface_to_volume_ratio': 6.911930751683295,
      'cluster_size_points': 136}},
    {'id': '6c8f41a572a8481e97e2ca459e1ce7ae',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-295.174, 158.985, 24.21],
       [-295.86400000000003, 159.358, 23.469],
       [-295.148, 158.874, 24.646],
       [-295.12, 158.859, 24.373],
       [-295.197, 158.90200000000002, 23.762],
       [-295.718, 159.183, 23.444],
       [-295.261, 158.802, 23.911],
       [-295.423, 158.89000000000001, 23.657],
       [-297.01, 159.343, 24.014],
       [-296.88, 159.198, 25.565],
       [-296.976, 159.25, 25.484],
       [-297.009, 159.155, 25.66],
       [-297.07, 159.09, 25.71],
       [-297.66, 159.327, 25.748],
       [-298.23900000000003, 159.639, 24.82],
       [-296.884, 158.68800000000002, 24.041],
       [-298.943, 159.70000000000002, 25.117],
       [-297.205, 158.65800000000002, 24.162],
       [-298.795, 159.38, 26.469],
       [-299.2, 159.597, 25.259],
       [-299.252, 159.425, 25.98],
       [-299.398, 159.43, 25.401],
       [-299.234, 159.343, 25.293],
       [-298.702, 158.972, 24.990000000000002],
       [-299.238, 159.123, 26.074],
       [-299.258, 159.026, 25.333000000000002],
       [-299.222, 158.951, 26.007]]},
     'customEntityData': {'X_centroid': np.float64(-297.27192387543255),
      'Y_centroid': np.float64(159.12672318339102),
      'Z_centroid': np.float64(24.804304498269897),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.9425470633905879),
      'volume': 2.826725918166677,
      'surface_area': 16.121707571720364,
      'surface_to_volume_ratio': 5.703314731757362,
      'cluster_size_points': 289}},
    {'id': '2a38d061ab884a1ab8ad3a918b8dcd37',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-312.954, 162.28900000000002, 37.707],
       [-312.722, 162.17000000000002, 36.928],
       [-312.677, 162.05100000000002, 36.824],
       [-312.757, 162.093, 36.735],
       [-312.423, 161.828, 38.318],
       [-312.281, 161.756, 37.836],
       [-312.729, 161.991, 36.858000000000004],
       [-312.78700000000003, 162.02, 36.771],
       [-312.258, 161.615, 38.072],
       [-312.281, 161.627, 37.986000000000004],
       [-313.966, 162.501, 37.625],
       [-313.31600000000003, 161.983, 39.794000000000004],
       [-312.392, 161.51, 38.082],
       [-312.511, 161.44, 38.613],
       [-312.553, 161.338, 38.462],
       [-313.934, 161.986, 37.543],
       [-313.85, 161.814, 39.267],
       [-313.89300000000003, 161.731, 39.289],
       [-314.036, 161.809, 37.709],
       [-313.968, 161.696, 37.643],
       [-314.085, 161.613, 39.088],
       [-314.132, 161.546, 39.050000000000004],
       [-313.334, 161.138, 38.203],
       [-313.533, 161.151, 38.289],
       [-313.337, 161.05100000000002, 37.898],
       [-313.351, 161.058, 37.801],
       [-313.628, 161.073, 37.861000000000004],
       [-313.654, 161.086, 37.775],
       [-314.195, 161.266, 38.019],
       [-313.911, 161.12, 37.89]]},
     'customEntityData': {'X_centroid': np.float64(-313.2723025641025),
      'Y_centroid': np.float64(161.65669743589743),
      'Z_centroid': np.float64(38.3),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(2.8041589355526857),
      'volume': 2.4609652386666814,
      'surface_area': 10.875997135544576,
      'surface_to_volume_ratio': 4.419402990607478,
      'cluster_size_points': 195}},
    {'id': '99787f81405e48a8a14fe26dc77f02d6',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-307.561, 158.596, 28.646],
       [-307.575, 158.603, 28.560000000000002],
       [-307.962, 158.70000000000002, 28.516000000000002],
       [-307.656, 158.328, 28.0],
       [-307.84000000000003, 158.026, 27.798000000000002],
       [-306.919, 157.041, 27.621000000000002],
       [-306.748, 156.769, 27.908],
       [-307.157, 156.85, 27.361],
       [-306.866, 156.702, 27.244],
       [-307.36400000000003, 156.636, 29.719],
       [-310.04, 157.58, 29.277],
       [-306.873, 155.919, 28.712],
       [-314.362, 159.466, 36.67],
       [-312.785, 158.675, 32.578],
       [-314.406, 159.417, 36.682],
       [-314.569, 158.862, 36.099000000000004],
       [-306.892, 154.711, 27.076],
       [-311.616, 156.955, 30.134],
       [-307.011, 154.641, 27.048000000000002],
       [-310.988, 156.516, 38.007],
       [-307.63100000000003, 154.851, 27.186],
       [-310.505, 156.187, 37.81],
       [-313.886, 157.899, 33.081],
       [-307.765, 154.832, 27.225],
       [-310.051, 155.84, 37.437],
       [-308.002, 154.83100000000002, 27.311],
       [-315.463, 158.254, 38.332],
       [-315.043, 158.049, 36.219],
       [-315.502, 158.149, 38.335],
       [-315.415, 157.493, 41.421],
       [-315.361, 157.372, 41.475],
       [-309.317, 154.165, 27.73],
       [-308.02, 153.148, 27.217000000000002],
       [-308.237, 153.13, 27.284],
       [-307.312, 152.67000000000002, 27.115000000000002],
       [-307.814, 152.735, 27.2],
       [-310.908, 154.124, 40.645],
       [-310.771, 154.05700000000002, 40.442],
       [-310.664, 153.901, 40.372],
       [-312.505, 154.757, 29.185000000000002],
       [-316.456, 156.454, 41.464],
       [-309.436, 152.991, 37.784],
       [-309.403, 152.894, 37.748],
       [-316.267, 156.145, 41.951],
       [-307.688, 151.943, 27.228],
       [-306.813, 151.415, 27.275000000000002],
       [-310.55, 153.153, 40.685],
       [-312.451, 153.945, 42.868],
       [-312.31600000000003, 153.913, 28.958000000000002],
       [-311.925, 153.598, 42.913000000000004],
       [-312.02, 153.566, 42.991],
       [-312.582, 153.876, 29.074],
       [-306.959, 150.979, 27.159],
       [-310.782, 152.717, 41.025],
       [-310.657, 152.656, 40.824],
       [-316.672, 155.546, 42.067],
       [-316.86, 155.639, 41.912],
       [-313.074, 153.649, 42.959],
       [-307.021, 150.239, 27.419],
       [-307.19100000000003, 150.192, 28.059],
       [-307.125, 150.161, 27.777],
       [-316.956, 154.83100000000002, 41.163000000000004],
       [-316.901, 154.805, 40.886],
       [-316.987, 153.713, 41.306],
       [-310.633, 150.54, 37.646],
       [-316.991, 153.519, 41.396],
       [-316.907, 152.752, 40.501],
       [-313.666, 150.994, 42.925000000000004],
       [-314.332, 151.094, 43.601],
       [-314.153, 151.008, 43.479],
       [-312.726, 149.407, 39.589],
       [-316.708, 151.238, 42.393],
       [-316.743, 151.255, 42.12],
       [-316.899, 151.198, 40.7],
       [-315.426, 150.086, 43.619],
       [-315.451, 149.89600000000002, 43.642],
       [-316.64300000000003, 150.049, 42.182],
       [-314.791, 148.966, 42.361000000000004],
       [-314.829, 149.012, 30.137],
       [-314.794, 148.87, 42.28],
       [-314.87, 148.866, 30.216],
       [-316.53000000000003, 149.315, 41.296],
       [-316.0, 148.332, 41.748],
       [-314.782, 147.474, 29.51],
       [-314.848, 147.407, 29.555],
       [-314.858, 147.331, 29.546],
       [-314.956, 147.262, 29.848],
       [-314.92900000000003, 147.248, 29.753],
       [-314.796, 147.186, 29.459],
       [-314.978, 147.15800000000002, 29.946],
       [-314.95300000000003, 147.147, 29.847],
       [-314.716, 147.035, 29.363],
       [-315.053, 146.903, 30.459],
       [-315.048, 146.80700000000002, 30.607],
       [-315.022, 146.79500000000002, 30.424],
       [-315.048, 146.578, 32.184],
       [-314.587, 146.369, 29.464000000000002],
       [-314.921, 146.337, 32.676],
       [-314.697, 145.924, 32.334],
       [-314.697, 145.924, 32.242],
       [-314.612, 145.889, 30.483],
       [-314.507, 145.84, 30.288],
       [-314.428, 145.804, 30.193],
       [-314.598, 145.766, 32.634],
       [-314.6, 145.767, 32.538000000000004],
       [-314.511, 145.657, 32.703],
       [-314.516, 145.659, 32.52],
       [-314.351, 145.448, 32.458],
       [-314.145, 145.264, 32.726],
       [-314.174, 145.278, 32.641],
       [-314.237, 145.30700000000002, 32.556]]},
     'customEntityData': {'X_centroid': np.float64(-312.31687710982663),
      'Y_centroid': np.float64(152.65071289017342),
      'Z_centroid': np.float64(34.91100196531792),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(2.5251665656530253),
      'volume': 819.5216101473349,
      'surface_area': 537.401445348895,
      'surface_to_volume_ratio': 0.65575018241713,
      'cluster_size_points': 17300}},
    {'id': 'fa24053cd73045578e3aab02d259df57',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-313.916, 160.779, 43.561],
       [-313.954, 160.798, 43.472],
       [-314.106, 160.803, 43.58],
       [-314.145, 160.702, 43.47],
       [-314.377, 160.821, 43.408],
       [-313.903, 160.389, 43.646],
       [-315.03700000000003, 160.74, 43.849000000000004],
       [-315.365, 160.829, 43.776],
       [-314.31, 160.141, 44.375],
       [-315.456, 160.728, 43.874],
       [-313.933, 159.856, 44.081],
       [-315.789, 160.804, 43.298],
       [-313.873, 159.75, 44.691],
       [-314.98900000000003, 160.007, 44.514],
       [-314.034, 159.27700000000002, 44.616],
       [-314.141, 159.13400000000001, 44.591],
       [-316.26300000000003, 160.084, 44.263],
       [-314.883, 159.321, 44.944],
       [-314.778, 159.135, 44.911],
       [-316.37, 159.845, 44.292],
       [-314.745, 158.927, 44.391],
       [-315.66200000000003, 159.279, 44.766],
       [-316.371, 159.639, 44.397],
       [-316.361, 159.63400000000001, 44.296],
       [-314.777, 158.642, 44.829],
       [-314.686, 158.597, 44.726],
       [-314.926, 158.58700000000002, 44.717],
       [-315.444, 158.848, 44.696]]},
     'customEntityData': {'X_centroid': np.float64(-314.88012735849054),
      'Y_centroid': np.float64(159.99297641509438),
      'Z_centroid': np.float64(44.16728301886793),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(2.0021091089167817),
      'volume': 2.065691222166656,
      'surface_area': 11.93036972451057,
      'surface_to_volume_ratio': 5.775485511332657,
      'cluster_size_points': 212}},
    {'id': '0c765b8df5af4098b0cff46f8b811c66',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-314.479, 157.741, 44.969],
       [-314.07800000000003, 157.416, 45.371],
       [-314.041, 157.349, 45.296],
       [-314.116, 157.388, 45.204],
       [-315.186, 157.673, 44.844],
       [-314.148, 156.952, 45.258],
       [-315.926, 157.846, 43.911],
       [-314.687, 157.022, 45.801],
       [-314.984, 157.172, 45.472],
       [-314.51800000000003, 156.709, 45.954],
       [-314.408, 156.655, 45.844],
       [-314.413, 156.58, 45.821],
       [-315.841, 157.294, 44.611000000000004],
       [-315.897, 157.32500000000002, 44.147],
       [-314.938, 156.541, 45.889],
       [-315.752, 156.719, 45.198],
       [-314.943, 156.226, 45.974000000000004],
       [-315.527, 156.517, 45.686],
       [-315.72700000000003, 156.617, 45.332],
       [-314.776, 156.148, 44.259],
       [-314.961, 156.124, 44.183],
       [-314.999, 156.03900000000002, 44.311],
       [-315.175, 156.041, 44.213],
       [-315.921, 156.18, 44.783],
       [-315.562, 155.915, 45.917],
       [-315.53700000000003, 155.773, 45.904],
       [-315.418, 155.714, 45.788000000000004],
       [-315.474, 155.645, 45.743],
       [-315.432, 155.547, 45.464],
       [-315.654, 155.516, 45.56]]},
     'customEntityData': {'X_centroid': np.float64(-315.2809711538461),
      'Y_centroid': np.float64(156.9241826923077),
      'Z_centroid': np.float64(44.96534134615385),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(1.32726862854593),
      'volume': 2.8133877074999907,
      'surface_area': 11.846482144276484,
      'surface_to_volume_ratio': 4.210753502866267,
      'cluster_size_points': 208}},
    {'id': 'ac17010845a34bdfb8e40545df1913ba',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-315.473, 155.567, 45.749],
       [-315.546, 155.463, 45.454],
       [-316.283, 155.66, 44.99],
       [-316.428, 155.593, 45.066],
       [-316.426, 155.593, 44.971000000000004],
       [-316.531, 155.53300000000002, 44.984],
       [-316.55400000000003, 155.479, 44.972],
       [-316.647, 155.39600000000002, 45.329],
       [-316.668, 155.40800000000002, 45.053000000000004],
       [-316.034, 154.979, 45.346000000000004],
       [-316.7, 155.30700000000002, 45.063],
       [-316.009, 154.898, 45.392],
       [-316.178, 154.981, 45.321],
       [-316.75600000000003, 155.26500000000001, 45.215],
       [-316.70300000000003, 155.239, 45.118],
       [-316.266, 154.89000000000001, 45.385],
       [-316.117, 154.726, 45.688],
       [-316.218, 154.776, 45.51],
       [-316.186, 154.681, 45.657000000000004],
       [-316.729, 154.502, 45.846000000000004]]},
     'customEntityData': {'X_centroid': np.float64(-316.18301086956524),
      'Y_centroid': np.float64(155.20483695652175),
      'Z_centroid': np.float64(45.393130434782606),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(2.8510693287231654),
      'volume': 0.2701234799999917,
      'surface_area': 2.899379284467333,
      'surface_to_volume_ratio': 10.733532991902155,
      'cluster_size_points': 92}},
    {'id': 'bf591c8ed2594b939516f1936c4c7ac8',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-309.169, 149.846, 30.809],
       [-309.206, 149.773, 30.881],
       [-308.927, 149.639, 30.402],
       [-309.374, 149.745, 30.959],
       [-309.272, 149.697, 30.310000000000002],
       [-309.127, 149.627, 30.026],
       [-309.284, 149.604, 29.979],
       [-309.26800000000003, 149.29, 31.081],
       [-309.799, 149.037, 31.194],
       [-310.087, 149.177, 30.583000000000002],
       [-310.397, 148.841, 30.463],
       [-310.229, 148.648, 30.703],
       [-310.308, 148.584, 30.204],
       [-310.399, 148.026, 30.497]]},
     'customEntityData': {'X_centroid': np.float64(-309.5853455882353),
      'Y_centroid': np.float64(149.23238970588238),
      'Z_centroid': np.float64(30.64193382352941),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(2.862771092919566),
      'volume': 0.5362552044999642,
      'surface_area': 4.426737053428847,
      'surface_to_volume_ratio': 8.254907395363363,
      'cluster_size_points': 136}},
    {'id': 'd20f5ec4165746009c89941e8c0b0958',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-307.972, 148.288, 28.598],
       [-307.994, 148.298, 28.514],
       [-307.307, 147.968, 28.183],
       [-308.548, 147.746, 27.762],
       [-308.371, 147.577, 29.622],
       [-307.84000000000003, 147.322, 29.212],
       [-308.57, 147.675, 27.742],
       [-308.45, 147.385, 29.693],
       [-307.938, 147.143, 29.191],
       [-308.634, 147.279, 29.733],
       [-308.23900000000003, 146.998, 29.023],
       [-308.383, 147.068, 28.406000000000002],
       [-308.406, 147.006, 28.457],
       [-308.815, 146.975, 29.154],
       [-308.52, 146.836, 28.581],
       [-308.846, 146.903, 29.027],
       [-308.609, 146.791, 28.642],
       [-308.879, 146.805, 28.909],
       [-308.838, 146.786, 28.815]]},
     'customEntityData': {'X_centroid': np.float64(-308.25989285714286),
      'Y_centroid': np.float64(147.3622142857143),
      'Z_centroid': np.float64(28.701294642857142),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.983470829695423),
      'volume': 0.9309956646666872,
      'surface_area': 6.073185879354333,
      'surface_to_volume_ratio': 6.523323480274895,
      'cluster_size_points': 112}},
    {'id': '9d400e1078244664a1e7b978abeb6a7a',
     'type': 'unknown',
     'dateTime': '2024-08-26T01:00:06',
     'geometry': {'type': 'Polygon',
      'coordinates': [[-266.012, 126.552, 6.884],
       [-266.514, 126.791, 6.744],
       [-266.153, 126.62, 6.658],
       [-266.053, 126.572, 6.574],
       [-266.073, 126.334, 6.632000000000001],
       [-266.916, 126.735, 5.488],
       [-266.395, 126.375, 6.132000000000001],
       [-267.619, 126.95400000000001, 5.926],
       [-267.858, 126.979, 6.026],
       [-267.803, 126.953, 5.9430000000000005],
       [-267.164, 126.65, 5.621],
       [-268.359, 127.157, 7.6690000000000005],
       [-268.412, 126.985, 7.8340000000000005],
       [-268.291, 126.928, 7.752],
       [-267.922, 126.69500000000001, 5.923],
       [-268.952, 127.061, 7.21],
       [-268.613, 126.901, 6.4990000000000006],
       [-268.561, 126.87700000000001, 6.416],
       [-268.48900000000003, 126.843, 6.336],
       [-268.916, 126.967, 7.094],
       [-268.673, 126.786, 7.455],
       [-268.918, 126.902, 7.069],
       [-267.76, 126.355, 6.961],
       [-267.77, 126.36, 6.8870000000000005],
       [-268.565, 126.736, 6.672],
       [-268.75100000000003, 126.706, 7.28],
       [-268.793, 126.726, 7.206]]},
     'customEntityData': {'X_centroid': np.float64(-267.6364581005587),
      'Y_centroid': np.float64(126.71088826815644),
      'Z_centroid': np.float64(6.727787709497207),
      'm3c2_magnitude_abs_average_per_cluster': np.float64(0.9408999777651242),
      'volume': 1.4372456621666572,
      'surface_area': 9.430714044463242,
      'surface_to_volume_ratio': 6.561657685051823,
      'cluster_size_points': 179}}]},
  {'backgroundImageData': {},
   'startDateTime': '2024-08-26T01:00:06',
   'endDateTime': '2024-08-26T02:00:05',
   'geoObjects': None},
  {'backgroundImageData': {},
   'startDateTime': '2024-08-26T02:00:05',
   'endDateTime': '2024-08-26T03:00:05',
   'geoObjects': None}]}

In [9]:
project_name = configuration["project_setting"]["project_name"]
list_observation_projection = []

for epoch_id, observation in enumerate(observations['observations']):
    if observation['geoObjects'] is None:
        list_observation_projection.append(None)
        continue
    background_projection = list_background_projections[epoch_id]

    observation_projection = projection.ProjectChange(observation=observation,
                            project_name=f"{project_name}_{epoch_id}_{epoch_id+1}",
                            projected_image_path=background_projection.bg_image_filename[0],
                            projected_events_folder="./out",
                            epsg=None)

    observation_projection.project_change()
    list_observation_projection.append(observation_projection)

Display the rockfall event in the study site

In [ ]:
from PIL import Image, ImageDraw
from shapely.geometry import shape, mapping
import json

# branch_crop_box = (130, 1170, 1600, 2000) # (left, upper, right, lower)
frames = []
gif_path = "../docs/img/rockfall_projections_plus_observations.gif"

for enum, img in enumerate(images[1:]):
    frm = Image.open(img).convert("RGB")
    # frm = frm.crop(branch_crop_box)
    draw = ImageDraw.Draw(frm)

    observation_projection = list_observation_projection[enum]

    # Load geojson
    if observation_projection is not None:
        with open(observation_projection.geojson_name, 'r') as f:
            geojson_data = json.load(f)

        for feature in geojson_data["features"]:
            coords = feature["geometry"]['coordinates'][0]
            coords = np.array(coords).reshape((1, len(coords)*2))[0]
            draw.polygon(coords, fill="black", outline=2)

    frames.append(frm)
    

frames[0].save(
    gif_path,
    save_all=True,
    append_images=frames[1:],
    duration=500,
    loop=0
)


# Bust cache with a unique query string
gif_display_path = gif_path + "?v=" + str(os.path.getmtime(gif_path))
HTML(f"""
<img src="{gif_display_path}" style="height:5in;" />
""")